In [1]:
import tensorflow as tf
import keras
import numpy as np

from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.models import load_model

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Define input and output

In [4]:
modelname = "myModel"

input_path = "./"
input_file = modelname + ".h5"

output_path = "/var/www/html/tutorial-data/"
output_file = modelname + ".pb"

output_node_prefix = "output_node"

In [5]:
K.set_learning_phase(0)
net_model = load_model(input_path + input_file)

num_output = 1
pred = [None]*num_output
pred_node_names = [None]*num_output
for i in range(num_output):
    pred_node_names[i] = output_node_prefix+str(i)
    pred[i] = tf.identity(net_model.outputs[i], name=pred_node_names[i])
print('output nodes names are: ', pred_node_names)
output_node_prefix = pred_node_names[0]

output nodes names are:  ['output_node0']


In [6]:
sess = K.get_session()

## Export model

In [7]:
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), pred_node_names)

graph_io.write_graph(constant_graph, output_path, output_file, as_text=False)

print('Saved the freezed graph at: ', (output_path + output_file))

INFO:tensorflow:Froze 4 variables.
Converted 4 variables to const ops.
Saved the freezed graph at:  /var/www/html/tutorial-data/myModel.pb


## Show input and output node

In [ ]:
g = tf.GraphDef()
g.ParseFromString(open(output_path + output_file, "rb").read())
s = ""
for n in g.node:
    s =s + str(n.name) + "\n"

print(s)